In [ ]:
# Cell 1: Import Libraries and Load Tokenizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AdamW, get_scheduler
from datasets import load_dataset
import pandas as pd
import wandb
from torch.cuda.amp import autocast, GradScaler
import gc
from tqdm import tqdm

# Initialize WandB in offline mode
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api_key")

wandb.login(key=secret_value_0)
wandb.init(project="DeBERTa-Fake-News", config={"epochs": 3, "batch_size": 32, "lr": 1e-5}, mode="offline")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# Free GPU cache
torch.cuda.empty_cache()
gc.collect()

# Load DeBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

In [2]:
# Cell 2: Load Datasets
# Load dataset from Hugging Face for pre-training
ds = load_dataset("Salesforce/wikitext", "wikitext-103-v1", split="train")
pretrain_texts = ds["text"][:int(0.05 * len(ds["text"]))]  # Use 10% of Wikitext-103
pretrain_df = pd.DataFrame({"text": pretrain_texts, "label": [0] * len(pretrain_texts)})

# Load dataset from CSV (WELFake) for fine-tuning
welfake_df = pd.read_csv("/kaggle/input/welfake-dataset-for-fake-news/WELFake_Dataset.csv", usecols=["text", "label"], dtype={"label": str})

# Ensure labels are numeric & clean
welfake_df["label"] = welfake_df["label"].str.replace(r"[^\d]", "", regex=True).str.strip()
welfake_df = welfake_df[welfake_df["label"] != ""]  # Remove empty values
welfake_df["label"] = welfake_df["label"].astype(int)  # Convert to int for PyTorch
print("Data loading complete!")

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/722k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/655k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Data loading complete!


In [3]:
# Cell 3: Define Dataset Classes
class MLMDataset(Dataset):
    def __init__(self, texts, max_len=128):
        self.encodings = tokenizer(texts, padding="max_length", truncation=True, max_length=max_len, return_tensors="pt")
        self.labels = self.encodings.input_ids.clone()
        rand = torch.rand(self.labels.shape)
        mask_arr = (rand < 0.15) * (self.labels != tokenizer.pad_token_id) * (self.labels != tokenizer.cls_token_id) * (self.labels != tokenizer.sep_token_id)
        self.encodings.input_ids[mask_arr] = tokenizer.mask_token_id
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}, self.labels[idx]

class FakeNewsDataset(Dataset):
    def __init__(self, df, max_len=256):
        self.labels = torch.tensor(df["label"].values, dtype=torch.long)
        self.encodings = tokenizer(df["text"].astype(str).tolist(), padding="max_length", truncation=True, 
                                   max_length=max_len, return_tensors="pt")
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items() if key != "token_type_ids"}, self.labels[idx]

In [4]:
# Cell 4: Define DeBERTa Model
class DeBERTaClassifier(nn.Module):
    def __init__(self, model_name="microsoft/deberta-v3-base", num_labels=2):
        super(DeBERTaClassifier, self).__init__()
        self.deberta = AutoModel.from_pretrained(model_name)
        self.deberta.gradient_checkpointing_enable()
        self.classifier = nn.Linear(self.deberta.config.hidden_size, num_labels)
        self.mlm_head = nn.Linear(self.deberta.config.hidden_size, self.deberta.config.vocab_size)  # MLM head for pretraining
    
    def forward(self, input_ids, attention_mask, mlm=False):
        outputs = self.deberta(input_ids=input_ids, attention_mask=attention_mask)
        if mlm:
            return self.mlm_head(outputs.last_hidden_state)
        logits = self.classifier(outputs.last_hidden_state[:, 0, :])
        return logits

In [10]:
# Cell 5: Initialize Model, Optimizer, and Scheduler
model = DeBERTaClassifier().to(device)
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

# Learning rate scheduler
num_training_steps = 3 * len(welfake_df) // 32  # Adjusted for batch size 32
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

scaler = GradScaler()

# Log model details to WandB
wandb.watch(model, log="all")

<ipython-input-10-9e1bdd0616b0>:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [6]:
# Check if multiple GPUs are available
multi_gpu = torch.cuda.device_count() > 1
if multi_gpu:
    print(f"Using {torch.cuda.device_count()} GPUs!")

Using 2 GPUs!


In [11]:
# Cell 6: Pre-training Loop
def pretrain_model(model, train_loader, epochs=1):
    model.train()
    mlm_criterion = nn.CrossEntropyLoss(ignore_index=-100)
    
    for epoch in range(epochs):
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Pretraining Epoch {epoch+1}")
        
        for batch in progress_bar:
            inputs, labels = batch
            inputs = {key: val.to(device) for key, val in inputs.items()}
            labels = labels.to(device)
            optimizer.zero_grad()
            
            with torch.amp.autocast("cuda", dtype=torch.bfloat16):
                outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], mlm=True)
                loss = mlm_criterion(outputs.view(-1, model.deberta.config.vocab_size), labels.view(-1))
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            total_loss += loss.item()
            progress_bar.set_postfix(loss=total_loss / len(train_loader))
        
        # Log loss per epoch to WandB
        wandb.log({"Pretrain Epoch Loss": total_loss / len(train_loader)})

In [ ]:
# Cell 7: Run Pre-training
pretrain_dataset = MLMDataset(pretrain_texts)
pretrain_loader = DataLoader(pretrain_dataset, batch_size=32, shuffle=True)
pretrain_model(model, pretrain_loader)

Pretraining Epoch 1:  72%|███████▏  | 2038/2815 [1:35:52<36:33,  2.82s/it, loss=1.84]  

In [14]:
save_path = "/kaggle/working/deberta_pretrained_checkpoint.pth"
torch.save(model.state_dict(), save_path)
print(f"Checkpoint saved at: {save_path}")

Checkpoint saved at: /kaggle/working/deberta_pretrained_checkpoint.pth


In [16]:
# Cell 8: Fine-tuning Loop
def fine_tune_model(model, train_loader):
    model.train()
    total_loss, correct, total = 0, 0, 0
    progress_bar = tqdm(train_loader, desc="Fine-tuning Epoch 1")
    
    for batch in progress_bar:
        inputs, labels = batch
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)
        
        optimizer.zero_grad()
        with torch.amp.autocast("cuda"):
            outputs = model(**inputs)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        lr_scheduler.step()
        
        # Compute Accuracy
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        
        total_loss += loss.item()
        progress_bar.set_postfix(loss=total_loss / (total + 1e-8), accuracy=correct / total)

    # Log fine-tuning loss and accuracy to WandB
    wandb.log({"Fine-tune Loss": total_loss / len(train_loader), "Fine-tune Accuracy": correct / total})

In [17]:
# Cell 9: Run Fine-tuning
train_dataset = FakeNewsDataset(welfake_df)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

fine_tune_model(model, train_loader)

Fine-tuning Epoch 1: 100%|██████████| 2255/2255 [36:17<00:00,  1.04it/s, accuracy=0.974, loss=0.00176]


In [18]:
# Cell 10: Save Fine-Tuned Model
model_save_path = "./deberta_finetuned.pth"
torch.save(model.state_dict(), model_save_path)

# Log model checkpoint to WandB
wandb.save(model_save_path)

print(f"Model saved to {model_save_path}")

Model saved to ./deberta_finetuned.pth


In [ ]:
# Cell 11: Model Evaluation and Testing
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, matthews_corrcoef, confusion_matrix

def evaluate_model(model, test_loader):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    all_preds, all_labels, all_probs = [], [], []
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating Model"):
            inputs, labels = batch
            inputs = {key: val.to(device) for key, val in inputs.items()}
            labels = labels.to(device)
            
            outputs = model(**inputs)
            loss = criterion(outputs, labels)

            # Compute predictions & probabilities
            probs = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()  # Probability of class 1
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            labels = labels.cpu().numpy()

            # Store batch results
            all_preds.extend(preds)
            all_labels.extend(labels)
            all_probs.extend(probs)

            correct += (preds == labels).sum()
            total += labels.shape[0]
            total_loss += loss.item()
    
    avg_loss = total_loss / len(test_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)
    roc_auc = roc_auc_score(all_labels, all_probs)
    logloss = log_loss(all_labels, all_probs)
    mcc = matthews_corrcoef(all_labels, all_preds)
    cm = confusion_matrix(all_labels, all_preds)

    print(f"Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4%}, F1 Score: {f1:.4f}")

    # Log evaluation results to WandB
    wandb.log({
        "Test Loss": avg_loss,
        "Test Accuracy": accuracy,
        "Test Precision": precision,
        "Test Recall": recall,
        "Test F1 Score": f1,
        "Test ROC AUC": roc_auc,
        "Test Log Loss": logloss,
        "Test MCC": mcc,
        "Test Confusion Matrix": cm.tolist()
    })

# Run Evaluation
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
evaluate_model(model, test_loader)

Evaluating Model:  66%|██████▌   | 1493/2255 [16:48<08:33,  1.49it/s]